In [ ]:
%pip install -q -r requirements.txt

In [ ]:
import importlib
import functions.core_functions as core_functions
import dask
import dask.dataframe as dd
from dask.dataframe.utils import assert_eq
import pandas as pd
import numpy as np
import pandas_gbq
import json
import gc
import cudf
import os
from datetime import datetime, timedelta

dask.config.set({"dataframe.backend": "cudf"})

importlib.reload(core_functions)

In [ ]:
resp = {}
resp = core_functions.initialize_clients(service_account_secret_name='SA_N90_CORE_APPS')

config = resp.get('config')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
bigquery_client = resp.get('clients').get('bigquery_client')
storage_client = resp.get('clients').get('storage_client')

In [ ]:
gclid_sql = f"""
  CREATE TEMP TABLE sessionGclids AS (
    SELECT 
        gclid,
        MIN(created_time) AS created_time,
        DATE(MIN(created_time)) AS date,
        EXTRACT(YEAR FROM MIN(created_time)) AS year,
        EXTRACT(MONTH FROM MIN(created_time)) AS month,
        EXTRACT(DAY FROM MIN(created_time)) AS day
    FROM `next90-core-applications.omniData.activity_sessions`
    WHERE gclid IS NOT NULL 
    GROUP BY gclid
    ORDER BY created_time
  );
SELECT * FROM `next90-core-applications.next90_google_ads_transfer_new.ads_ClickStats_8417741864`
join sessionGclids
on gclid = click_view_gclid
AND (date = segments_date or date = DATE_ADD(segments_date, INTERVAL 1 DAY))
"""

gclid_base_df = core_functions.fetch_gbq_data(gclid_sql, bigquery_client=bigquery_client)
gclid_base_df_working = gclid_base_df.copy()

In [ ]:
gclid_base_df_working.head()

In [ ]:
core_functions.show_more_dataframe()
gclid_base_df_working['kw_criteria_id'] = gclid_base_df_working['click_view_keyword'].str.split('~').str[1]
gclid_base_df_working['ad_id'] = gclid_base_df_working['click_view_ad_group_ad'].str.split('~').str[1]
gclid_base_df_working['click_city_presence_location'] = gclid_base_df_working['click_view_location_of_presence_city'].str.split('/').str[1]
gclid_base_df_working['click_metro_presence_location'] = gclid_base_df_working['click_view_location_of_presence_metro'].str.split('/').str[1]
gclid_base_df_working['click_most_specific_presence_location'] = gclid_base_df_working['click_view_location_of_presence_most_specific'].str.split('/').str[1]
gclid_base_df_working['view_timestamp'] = gclid_base_df_working['created_time'] - timedelta(seconds=1)
gclid_base_df_working.loc[~gclid_base_df['click_view_keyword'].isna()].head(20)

In [ ]:
gclid_base_df_working.loc[~gclid_base_df['click_view_keyword'].isna()].head(10)
customer_ids = gclid_base_df_working['customer_id'].unique()
min_date = gclid_base_df_working['date'].min()
min_date


In [ ]:
gclid_base_df_working.head(3)

In [ ]:
kw_stats_sql = f"""
SELECT * FROM `next90-core-applications.next90_google_ads_transfer_new.ads_KeywordStats_8417741864` 
WHERE segments_date >= '{min_date}'
AND customer_id IN ({','.join([str(x) for x in customer_ids])})
"""

kw_stats_df = core_functions.fetch_gbq_data(kw_stats_sql, bigquery_client=bigquery_client)
kw_stats_df.head(3)


In [ ]:
kw_stats_df['ad_group_criterion_criterion_id'] = kw_stats_df['ad_group_criterion_criterion_id'].astype(str)

In [ ]:
gclid_base_df_working2 = gclid_base_df_working.merge(kw_stats_df, how='left', left_on=['customer_id', 'kw_criteria_id', 'segments_date', 'segments_click_type','segments_ad_network_type','segments_device'], right_on=['customer_id', 'ad_group_criterion_criterion_id', 'segments_date','segments_click_type','segments_ad_network_type','segments_device'], suffixes=('', '_kw_stats'))

In [ ]:
gclid_base_df_working2.loc[~gclid_base_df_working2['metrics_average_cpc'].isna()].head()